# OpenEO GFMap: Local execution of framework

As the framework is also aiming for a local execution of OpenEO, let's test the local environment by loading the data from a STAC catalogue.

In [ ]:
import openeo
from pystac_client import Client

# Do not need a connection to a backend, but to rely on the STAC catalogue and do everything locally.

catalog = Client.open('https://services.terrascope.be/stac/')
collection = catalog.get_collection(collection_id='urn:eop:VITO:TERRASCOPE_S2_TOC_V2')
collection_href = collection.links[0].absolute_href

collection.id, collection.title

In [ ]:
collection_href

'https://services.terrascope.be/stac/collections/urn:eop:VITO:TERRASCOPE_S2_TOC_V2'

In [4]:
from openeo.local import LocalConnection
from datetime import datetime, timedelta

connection = LocalConnection('./resources/')

SPATIAL_EXTENT = {
    "west": 4.1488819,
    "south": 51.285309,
    "east": 4.1775545,
    "north": 51.3032902,
    "crs": "EPSG:4326"
}

# Summer 2023
TEMPORAL_EXTENT = ["2022-06-21", "2022-09-23"]

cube = connection.load_stac(
    'https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a',
    spatial_extent=SPATIAL_EXTENT,
    temporal_extent=TEMPORAL_EXTENT,
    bands=['green', 'blue', 'red', 'nir', 'scl']
)

cube = cube.execute()

/home/couchard/miniconda3/envs/gfmap/lib/python3.9/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(
/home/couchard/miniconda3/envs/gfmap/lib/python3.9/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [7]:
openeo.client_version()

'0.23.0'

In [ ]:
# Perform the scl mask dilation

# IS NOT WORKING: Process does not exists, need to find a replacement
cube = cube.process(
    'mask_scl_dilation',
    data=cube,
    scl_band_name='scl',
    kernel1_size=17,
    kernel2_size=77,
    mask1_values=[2, 4, 5, 6, 7],
    mask2_values=[3, 8, 9, 10, 11],
    erosion_kernel_size=3
).filter_bands(['green', 'blue', 'red', 'nir'])

# Perform temporal compositing
compositing_dates = []

date_format = '%Y-%m-%d'

start_date = datetime.strptime(TEMPORAL_EXTENT[0], date_format)
end_date = datetime.strptime(TEMPORAL_EXTENT[-1], date_format)

# Difference
n_days = (end_date - start_date).days
interval_days = 10

date = start_date
while date + timedelta(days=interval_days) < end_date:
    compositing_dates.append([
        date.strftime(date_format),
        (date + timedelta(days=interval_days)).strftime(date_format)
    ])
    date = date + timedelta(days=interval_days)

# IS NOT WORKING either
cube = cube.aggregate_temporal(intervals=compositing_dates, reducer='median')


# Interpolate linear

cube = cube.apply_dimension(dimension='time', process='array_interpolate_linear')

# download the data
cube = cube.execute()
cube